# Définition et Calcul des KPI

**Objectif** : Définir et calculer au moins 6 indicateurs de pilotage métier pertinents pour optimiser le réseau de services publics au Togo.


## 1. Import des Librairies et Chargement des Données Nettoyées

In [4]:
# Import des librairies nécessaires
import pandas as pd
import numpy as np
import sqlite3

# --- 1) Dataset final (utile pour vérifs rapides / dashboard) ---
data_path = '../data/cleaned_data/dataset_nettoye.csv'
df = pd.read_csv(data_path, encoding='utf-8')
print(f"✅ Dataset final chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes")

print("------------- ",df.duplicated())

# --- 2) Base SQLite en mémoire pour exécuter les KPI en SQL ---
conn = sqlite3.connect(':memory:')
df.to_sql('dataset_nettoye', conn, index=False, if_exists='replace')

# --- 3) Tables sources (grain correct) ---
# Remarque : le dataset final contient des duplications dues aux jointures.
# Pour des KPI fiables, on privilégie les tables au bon niveau (demandes / centres / communes / logs).

data_files = {
    'demandes_service_public': '../data/demandes_service_public.csv',
    'centres_service': '../data/centres_service.csv',
    'details_communes': '../data/details_communes.csv',
    'donnees_socioeconomiques': '../data/donnees_socioeconomiques.csv',
    'logs_activite': '../data/logs_activite.csv',
}

datasets = {}
for name, path in data_files.items():
    tmp = pd.read_csv(path, encoding='utf-8')
    datasets[name] = tmp
    tmp.to_sql(name, conn, index=False, if_exists='replace')
    print(f"✅ Table SQL chargée : {name} ({tmp.shape[0]} lignes, {tmp.shape[1]} colonnes)")

print("\n✅ SQLite prêt : dataset_nettoye + tables sources")

✅ Dataset final chargé : 5121 lignes, 55 colonnes
-------------  0       False
1       False
2       False
3       False
4       False
        ...  
5116    False
5117    False
5118    False
5119    False
5120    False
Length: 5121, dtype: bool
✅ Table SQL chargée : demandes_service_public (600 lignes, 16 colonnes)
✅ Table SQL chargée : centres_service (55 lignes, 16 colonnes)
✅ Table SQL chargée : details_communes (200 lignes, 13 colonnes)
✅ Table SQL chargée : donnees_socioeconomiques (115 lignes, 11 colonnes)
✅ Table SQL chargée : logs_activite (450 lignes, 14 colonnes)

✅ SQLite prêt : dataset_nettoye + tables sources


## 2. KPI 1 : Délai Moyen de Traitement (Performance Opérationnelle)

In [ ]:
"""
KPI 1 : Délai Moyen de Traitement des Demandes

Objectif : mesurer l'efficacité opérationnelle (temps moyen en jours pour traiter une demande). On ne prend que les demandes traitées, avec un délai strictement positif et ≤ 365 jours.
Approche : SQL uniquement (filtre + agrégation par région).
"""

sql_kpi1 = """
SELECT 
    'global' AS region,
    COUNT(*) AS nb_demandes_traitees,
    ROUND(AVG(delai_traitement_jours), 2) AS delai_moyen_jours
FROM demandes_service_public
WHERE LOWER(statut_demande) = 'traitee'
  AND delai_traitement_jours > 0
  AND delai_traitement_jours <= 365

UNION ALL

SELECT 
    region AS region,
    COUNT(*) AS nb_demandes_traitees,
    ROUND(AVG(delai_traitement_jours), 2) AS delai_moyen_jours
FROM demandes_service_public
WHERE LOWER(statut_demande) = 'traitee'
  AND delai_traitement_jours > 0
  AND delai_traitement_jours <= 365
GROUP BY region
ORDER BY CASE WHEN region = 'global' THEN 0 ELSE 1 END, delai_moyen_jours DESC;
"""

print("📝 Requête SQL (KPI 1 - Délai moyen) :")
print(sql_kpi1)
result_kpi1 = pd.read_sql_query(sql_kpi1, conn)
print("\n✅ Résultats KPI 1 :")
print(result_kpi1)


📊 Délai moyen de traitement (global) : 21.71 jours

📊 Délai moyen par région :
  - centrale : 19.3 jours
  - kara : 25.22 jours
  - maritime : 21.8 jours
  - plateaux : 21.78 jours
  - savanes : 22.3 jours

📝 Requête SQL équivalente :

SELECT 
    region_demandes AS region,
    COUNT(*) AS nb_demandes_traitees,
    ROUND(AVG(delai_traitement_jours), 2) AS delai_moyen_jours
FROM dataset_nettoye
WHERE LOWER(statut_demande) = 'traitee'
    AND delai_traitement_jours > 0
    AND delai_traitement_jours <= 365
GROUP BY region_demandes
ORDER BY delai_moyen_jours DESC;


✅ Résultats de la requête SQL :
     region  nb_demandes_traitees  delai_moyen_jours
0      kara                   288              25.22
1   savanes                   280              22.30
2  maritime                   492              21.80
3  plateaux                   204              21.78
4  centrale                   516              19.30


## 3. KPI 2 : Taux d'Utilisation de la Capacité (Performance Opérationnelle)

In [ ]:
"""
KPI 2 : Taux d'Utilisation de la Capacité des Centres

Pourquoi SQL ici ?
- Ce KPI est une agrégation "centre × période" : SUM(volumes) / capacité.
- On calcule sur les tables sources (grain correct), sinon un dataset joint peut dupliquer les lignes.

Règle :
Taux d'utilisation (%) = (Total traité sur la période / (Capacité journalière × nb de jours observés)) × 100
- Total traité : SUM(logs_activite.nombre_traite)
- Capacité journalière : centres_service.personnel_capacite_jour
- nb de jours observés : COUNT(DISTINCT logs_activite.date_operation)
"""

sql_kpi2 = """
WITH jours AS (
  SELECT
    centre_id,
    COUNT(DISTINCT date_operation) AS nb_jours
  FROM logs_activite
  WHERE personnel_present > 0
  GROUP BY centre_id
),
volumes AS (
  SELECT
    centre_id,
    SUM(nombre_traite) AS total_traite
  FROM logs_activite
  GROUP BY centre_id
),
capacites AS (
  SELECT
    centre_id,
    MAX(personnel_capacite_jour) AS capacite_jour,
    MAX(nom_centre) AS nom_centre,
    MAX(region) AS region,
    MAX(commune) AS commune
  FROM centres_service
  GROUP BY centre_id
)
SELECT
  c.centre_id,
  c.nom_centre,
  c.region,
  c.commune,
  v.total_traite,
  j.nb_jours,
  c.capacite_jour,
  ROUND( (v.total_traite * 100.0) / (c.capacite_jour * j.nb_jours), 2) AS taux_utilisation_pct
FROM capacites c
JOIN volumes v ON v.centre_id = c.centre_id
JOIN jours j ON j.centre_id = c.centre_id
WHERE c.capacite_jour > 0 AND j.nb_jours > 0
ORDER BY taux_utilisation_pct DESC;
"""

print("📝 Requête SQL (KPI 2 - Taux utilisation capacité) :")
print(sql_kpi2)

result_kpi2 = pd.read_sql_query(sql_kpi2, conn)
print("\n✅ Résultats KPI 2 (top 10 centres) :")
print(result_kpi2.head(10))

print("\n📊 Résumé KPI 2 :")
print(result_kpi2['taux_utilisation_pct'].describe())

# --- Note méthodo ---
# Si tu veux un KPI plus simple (sans notion de jours), tu peux aussi faire :
# taux = SUM(nombre_traite) / MAX(capacite_jour) * 100
# mais l'interprétation devient "charge cumulée" plutôt que "taux d'occupation".



📊 Taux d'utilisation de la capacité par centre/commune :
  centre_id  nombre_demandes  personnel_capacite_jour      nom_centre  \
0     ct002            12618                       63     centre kévé   
1     ct002            12618                       63     centre kévé   
2     ct002            12618                       63     centre kévé   
3     ct002            12618                       63     centre kévé   
4     ct002            12618                       63     centre kévé   
5     ct002            12618                       63     centre kévé   
6     ct002            12618                       63     centre kévé   
7     ct003            12136                       98    centre gando   
8     ct003            12136                       98    centre gando   
9     ct004            14033                       78  centre pagouda   

     commune  taux_utilisation_pct  
0     tsévié              20028.57  
1       lomé              20028.57  
2  adidogome              20

In [4]:
"""
KPI 3 : Distance Moyenne aux Centres de Service

Objectif métier :
- Évaluer l'accessibilité géographique des services publics
- Identifier les zones sous-desservies nécessitant de nouveaux centres
- Optimiser la localisation des centres pour réduire les distances

Description / Interprétation :
- KPI exprimé en kilomètres (km)
- Valeur cible : < 25 km (bonne accessibilité)
- Valeur alarmante : > 50 km (faible accessibilité)
- Permet d'identifier les déserts de services et de planifier l'implantation de nouveaux centres

Règle de calcul :
Distance moyenne = (Somme des distances entre communes et centres les plus proches) / (Nombre de communes)
- Utiliser les coordonnées GPS (latitude, longitude) pour calculer la distance à vol d'oiseau
- Pour chaque commune, identifier le centre le plus proche
- Formule de Haversine pour calculer la distance géodésique
"""

# Fonction pour calculer la distance Haversine
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calcule la distance entre deux points GPS en km (formule de Haversine)
    """
    from math import radians, sin, cos, sqrt, atan2
    
    R = 6371  # Rayon de la Terre en km
    
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = sin(dlat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    distance = R * c
    return distance

# Calcul en Python
def calculer_distance_moyenne(df):
    """
    Calcule la distance moyenne des communes aux centres les plus proches
    """
    # Extraire les coordonnées des communes (si disponibles)
    communes_coords = df[['commune', 'latitude', 'longitude']].drop_duplicates().dropna()
    
    # Extraire les coordonnées des centres
    if 'latitude' in df.columns and 'longitude' in df.columns:
        centres_coords = df[['commune', 'latitude', 'longitude']].drop_duplicates().dropna()
    else:
        # Utiliser les coordonnées des communes comme proxy
        centres_coords = communes_coords.copy()
    
    distances = []
    
    for idx, commune in communes_coords.iterrows():
        if pd.notna(commune['latitude']) and pd.notna(commune['longitude']):
            min_distance = float('inf')
            
            # Trouver le centre le plus proche
            for idx2, centre in centres_coords.iterrows():
                if pd.notna(centre['latitude']) and pd.notna(centre['longitude']):
                    dist = haversine_distance(
                        commune['latitude'], commune['longitude'],
                        centre['latitude'], centre['longitude']
                    )
                    if dist < min_distance:
                        min_distance = dist
            
            if min_distance != float('inf'):
                distances.append({
                    'commune': commune['commune'],
                    'distance_km': min_distance
                })
    
    if len(distances) == 0:
        return None
    
    df_distances = pd.DataFrame(distances)
    distance_moyenne = df_distances['distance_km'].mean()
    
    return distance_moyenne, df_distances

# Calcul
result = calculer_distance_moyenne(df)
if result:
    distance_moyenne, df_distances = result
    print(f"📊 Distance moyenne aux centres de service : {distance_moyenne:.2f} km")
    print(f"\n📊 Top 10 communes les plus éloignées :")
    print(df_distances.nlargest(10, 'distance_km'))
else:
    # Utiliser distance_capitale_km comme proxy si disponible
    if 'distance_capitale_km' in df.columns:
        distance_moyenne = df['distance_capitale_km'].mean()
        print(f"📊 Distance moyenne à la capitale (proxy) : {distance_moyenne:.2f} km")
    else:
        print("⚠️ Coordonnées GPS non disponibles pour le calcul précis")

# Requête SQL équivalente (approximation avec distance_capitale_km)
sql_query = """
SELECT 
    region_demandes AS region,
    COUNT(DISTINCT commune) AS nb_communes,
    ROUND(AVG(distance_capitale_km), 2) AS distance_moyenne_capitale_km
FROM dataset_nettoye
WHERE distance_capitale_km IS NOT NULL
GROUP BY region_demandes
ORDER BY distance_moyenne_capitale_km DESC;
"""

print("\n📝 Requête SQL équivalente (proxy avec distance à la capitale) :")
print(sql_query)

# Exécution de la requête SQL
try:
    result_sql = pd.read_sql_query(sql_query, conn)
    print("\n✅ Résultats de la requête SQL :")
    print(result_sql)
except Exception as e:
    print(f"\n⚠️ Erreur SQL : {e}")

📊 Distance moyenne à la capitale (proxy) : nan km

📝 Requête SQL équivalente (proxy avec distance à la capitale) :

SELECT 
    region_demandes AS region,
    COUNT(DISTINCT commune) AS nb_communes,
    ROUND(AVG(distance_capitale_km), 2) AS distance_moyenne_capitale_km
FROM dataset_nettoye
WHERE distance_capitale_km IS NOT NULL
GROUP BY region_demandes
ORDER BY distance_moyenne_capitale_km DESC;


✅ Résultats de la requête SQL :
Empty DataFrame
Columns: [region, nb_communes, distance_moyenne_capitale_km]
Index: []


In [5]:
"""
KPI 4 : Taux de Couverture Démographique

Objectif métier :
- Mesurer le pourcentage de la population ayant accès à un centre à moins de X km
- Identifier les zones sous-desservies démographiquement
- Prioriser les investissements dans les zones à forte densité de population

Description / Interprétation :
- KPI exprimé en pourcentage (%)
- Valeur cible : > 80% de la population à moins de 25 km d'un centre
- Valeur alarmante : < 60% (faible couverture)
- Permet d'évaluer l'équité d'accès aux services publics

Règle de calcul :
Taux de couverture = (Population dans un rayon de X km autour d'un centre / Population totale) × 100
- Rayon de référence : 25 km (distance acceptable pour un service public)
- Utiliser les données démographiques par commune
- Compter la population des communes à moins de 25 km d'au moins un centre
"""

# Calcul en Python
def calculer_taux_couverture(df, rayon_km=25):
    """
    Calcule le taux de couverture démographique dans un rayon donné
    
    Parameters:
    -----------
    df : DataFrame
        Dataset nettoyé
    rayon_km : float
        Rayon de couverture en km (défaut : 25 km)
    
    Returns:
    --------
    float : Taux de couverture en %
    """
    # Extraire population et coordonnées par commune
    if 'population' in df.columns and 'latitude' in df.columns and 'longitude' in df.columns:
        communes_pop = df[['commune', 'population', 'latitude', 'longitude']].drop_duplicates().dropna()
    elif 'population_densite' in df.columns:
        # Utiliser densité comme proxy si population absolue absente
        communes_pop = df[['commune', 'population_densite', 'latitude', 'longitude']].drop_duplicates().dropna()
        communes_pop.rename(columns={'population_densite': 'population'}, inplace=True)
    else:
        return None
    
    # Identifier les centres (communes avec centres)
    centres_coords = df[['commune', 'latitude', 'longitude']].drop_duplicates().dropna()
    
    if len(communes_pop) == 0 or len(centres_coords) == 0:
        return None
    
    population_totale = communes_pop['population'].sum()
    population_couverte = 0
    
    # Pour chaque commune, vérifier si elle est dans le rayon d'un centre
    for idx, commune in communes_pop.iterrows():
        if pd.notna(commune['latitude']) and pd.notna(commune['longitude']):
            couverte = False
            
            for idx2, centre in centres_coords.iterrows():
                if pd.notna(centre['latitude']) and pd.notna(centre['longitude']):
                    dist = haversine_distance(
                        commune['latitude'], commune['longitude'],
                        centre['latitude'], centre['longitude']
                    )
                    if dist <= rayon_km:
                        couverte = True
                        break
            
            if couverte:
                population_couverte += commune['population']
    
    if population_totale == 0:
        return None
    
    taux_couverture = (population_couverte / population_totale) * 100
    return taux_couverture

# Calcul
taux_couverture = calculer_taux_couverture(df, rayon_km=25)
if taux_couverture:
    print(f"📊 Taux de couverture démographique (rayon 25 km) : {taux_couverture:.2f}%")
else:
    print("⚠️ Données démographiques ou coordonnées GPS non disponibles")

# Requête SQL équivalente (approximation)
sql_query = """
SELECT 
    region_demandes AS region,
    SUM(population) AS population_totale,
    COUNT(DISTINCT commune) AS nb_communes,
    COUNT(DISTINCT CASE WHEN distance_capitale_km <= 25 THEN commune END) AS communes_couvertes,
    ROUND(
        (COUNT(DISTINCT CASE WHEN distance_capitale_km <= 25 THEN commune END) * 100.0 / 
         COUNT(DISTINCT commune)), 2
    ) AS taux_couverture_communes_pct
FROM dataset_nettoye
WHERE population IS NOT NULL
GROUP BY region_demandes
ORDER BY taux_couverture_communes_pct DESC;
"""

print("\n📝 Requête SQL équivalente (approximation) :")
print(sql_query)

# Exécution de la requête SQL
try:
    result_sql = pd.read_sql_query(sql_query, conn)
    print("\n✅ Résultats de la requête SQL :")
    print(result_sql)
except Exception as e:
    print(f"\n⚠️ Erreur SQL : {e}")

⚠️ Données démographiques ou coordonnées GPS non disponibles

📝 Requête SQL équivalente (approximation) :

SELECT 
    region_demandes AS region,
    SUM(population) AS population_totale,
    COUNT(DISTINCT commune) AS nb_communes,
    COUNT(DISTINCT CASE WHEN distance_capitale_km <= 25 THEN commune END) AS communes_couvertes,
    ROUND(
        (COUNT(DISTINCT CASE WHEN distance_capitale_km <= 25 THEN commune END) * 100.0 / 
         COUNT(DISTINCT commune)), 2
    ) AS taux_couverture_communes_pct
FROM dataset_nettoye
WHERE population IS NOT NULL
GROUP BY region_demandes
ORDER BY taux_couverture_communes_pct DESC;


✅ Résultats de la requête SQL :
Empty DataFrame
Columns: [region, population_totale, nb_communes, communes_couvertes, taux_couverture_communes_pct]
Index: []


## 6. KPI 5 : Taux de Rejet des Demandes (Qualité de Service)

In [ ]:
"""
KPI 5 : Taux de Rejet des Demandes (Qualité de service)

Pourquoi SQL ici ?
- C'est un ratio simple : (rejetées / total), avec déclinaisons (région, type_document).

Règle :
Taux de rejet (%) = 100 × [Nb demandes rejetées] / [Nb demandes total]
- Table source utilisée : demandes_service_public (grain = 1 ligne par demande)
"""

sql_kpi5 = """
SELECT
  'global' AS region,
  'tous' AS type_document,
  COUNT(*) AS total_demandes,
  SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) AS demandes_rejetees,
  ROUND(100.0 * SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) / COUNT(*), 2) AS taux_rejet_pct
FROM demandes_service_public

UNION ALL

SELECT
  region AS region,
  type_document AS type_document,
  COUNT(*) AS total_demandes,
  SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) AS demandes_rejetees,
  ROUND(100.0 * SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) / COUNT(*), 2) AS taux_rejet_pct
FROM demandes_service_public
GROUP BY region, type_document
ORDER BY CASE WHEN region = 'global' THEN 0 ELSE 1 END, taux_rejet_pct DESC;
"""

print("📝 Requête SQL (KPI 5 - Taux de rejet) :")
print(sql_kpi5)

result_kpi5 = pd.read_sql_query(sql_kpi5, conn)
print("\n✅ Résultats KPI 5 (aperçu) :")
print(result_kpi5.head(15))


📊 Taux de rejet global : 34.92%

📊 Taux de rejet par région :
  - centrale : 35.34%
  - kara : 30.77%
  - maritime : 37.93%
  - plateaux : 28.46%
  - savanes : 38.26%

📊 Taux de rejet par type de document :
  - acte de naissance : 39.76%
  - carte d'identité : 33.72%
  - casier judiciaire : 32.5%
  - certificat de nationalité : 32.25%
  - livre de famille : 33.67%
  - passeport : 36.86%

📝 Requête SQL équivalente :

SELECT 
    region_demandes AS region,
    type_document,
    COUNT(*) AS total_demandes,
    SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) AS demandes_rejetees,
    ROUND(
        (SUM(CASE WHEN LOWER(statut_demande) = 'rejetée' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)), 2
    ) AS taux_rejet_pct
FROM dataset_nettoye
GROUP BY region_demandes, type_document
ORDER BY taux_rejet_pct DESC;


✅ Résultats de la requête SQL :
      region              type_document  total_demandes  demandes_rejetees  \
0   centrale                  passeport             240     

## 7. KPI 6 : Indice de Répartition Régionale (Efficience)

In [ ]:
"""
KPI 6 : Indice de Répartition Régionale (Efficience / Équité)

Pourquoi SQL + un tout petit Python ?
- SQL calcule parfaitement les volumes par région.
- SQLite n'a pas STDDEV nativement (selon la build), donc on calcule le CV en Python à partir des volumes.

Règle :
CV = (écart-type des volumes régionaux) / (moyenne des volumes régionaux)
"""

# 1) SQL : volumes par région (table source demandes_service_public)
sql_kpi6_volumes = """
SELECT
  region AS region,
  SUM(nombre_demandes) AS volume_total
FROM demandes_service_public
WHERE region IS NOT NULL
GROUP BY region
ORDER BY volume_total DESC;
"""

print("📝 Requête SQL (KPI 6 - volumes par région) :")
print(sql_kpi6_volumes)

volumes_region = pd.read_sql_query(sql_kpi6_volumes, conn)
print("\n✅ Volumes par région :")
print(volumes_region)

# 2) Python (minimal) : calcul du coefficient de variation (CV)
vals = volumes_region['volume_total'].astype(float)
mean = vals.mean()
std = vals.std(ddof=1)  # écart-type échantillon
cv = std / mean if mean else np.nan

print(f"\n📊 KPI 6 - Coefficient de variation (CV) : {cv:.3f}")
if cv < 0.3:
    print("✅ Excellente équité")
elif cv < 0.5:
    print("⚠️ Équité modérée")
else:
    print("❌ Forte inéquité")

# On garde le résultat pour la synthèse
result_kpi6 = pd.DataFrame({'cv': [round(cv, 3)]})


📊 Indice de répartition régionale (CV) : 0.375

📊 Volumes de demandes par région :
  region_demandes  nombre_demandes
2        maritime           151416
0        centrale           148644
1            kara           112264
4         savanes            84952
3        plateaux            54920

💡 Interprétation : ⚠️ Équité modérée - quelques déséquilibres à corriger

📝 Requête SQL équivalente :

WITH volumes_region AS (
    SELECT 
        region_demandes AS region,
        SUM(nombre_demandes) AS volume_total
    FROM dataset_nettoye
    WHERE region_demandes IS NOT NULL
    GROUP BY region_demandes
),
stats AS (
    SELECT 
        AVG(volume_total) AS moyenne,
        STDDEV(volume_total) AS ecart_type
    FROM volumes_region
)
SELECT 
    vr.region,
    vr.volume_total,
    s.moyenne,
    s.ecart_type,
    ROUND((s.ecart_type / s.moyenne), 3) AS coefficient_variation
FROM volumes_region vr
CROSS JOIN stats s
ORDER BY vr.volume_total DESC;


⚠️ Erreur SQL : Execution failed on sql '
W

## 8. Synthèse des KPI Calculés

In [ ]:
# Synthèse (SQL-first)
print("=" * 80)
print("SYNTHÈSE DES KPI (SQL-first)")
print("=" * 80)

# KPI 1 (SQL)
try:
    kpi1_global = result_kpi1[result_kpi1['region'] == 'global'].iloc[0]['delai_moyen_jours']
    print(f"\n1) Délai moyen de traitement (jours) : {kpi1_global}")
except Exception:
    print("\n1) Délai moyen : résultat indisponible")

# KPI 2 (SQL)
try:
    kpi2_median = float(result_kpi2['taux_utilisation_pct'].median())
    print(f"2) Taux d'utilisation capacité (médiane centres) : {kpi2_median:.2f}%")
except Exception:
    print("2) Taux d'utilisation : résultat indisponible")

# KPI 3 / KPI 4 : restent en Python si on veut Haversine (sinon proxy SQL)
print("3) Distance moyenne : Python (Haversine) si données GPS exploitables / sinon proxy")
print("4) Couverture démographique : Python (Haversine) / sinon proxy")

# KPI 5 (SQL)
try:
    kpi5_global = result_kpi5[(result_kpi5['region'] == 'global') & (result_kpi5['type_document'] == 'tous')].iloc[0]['taux_rejet_pct']
    print(f"5) Taux de rejet global : {kpi5_global}%")
except Exception:
    print("5) Taux de rejet : résultat indisponible")

# KPI 6 (SQL + Python minimal)
try:
    print(f"6) Indice de répartition régionale (CV) : {float(result_kpi6['cv'].iloc[0]):.3f}")
except Exception:
    print("6) Indice de répartition : résultat indisponible")

print("\n" + "=" * 80)
print("✅ KPI calculés. Prochaine étape : 4.4 Dashboard Streamlit")
print("=" * 80)

SYNTHÈSE DES KPI CALCULÉS

1. Délai Moyen de Traitement : 21.71 jours
   ⚠️ Acceptable mais à améliorer

2. Taux d'Utilisation de la Capacité : 18361.31%
   ❌ Surcharge - renforcement nécessaire

3. Distance Moyenne : Données GPS insuffisantes

4. Taux de Couverture : Données insuffisantes

5. Taux de Rejet des Demandes : 34.92%
   ❌ Qualité insuffisante - amélioration nécessaire

6. Indice de Répartition Régionale (CV) : 0.375
   ⚠️ Équité modérée

✅ Tous les KPI ont été calculés avec succès
